# Supervisor (Polyglot)

In [ ]:
#!import ../nbs/Testing.dib

In [ ]:
#r @"../../../../../../../.nuget/packages/fsharp.control.asyncseq/3.2.1/lib/netstandard2.1/FSharp.Control.AsyncSeq.dll"
#r @"../../../../../../../.nuget/packages/system.reactive/5.0.0/lib/net5.0/System.Reactive.dll"
#r @"../../../../../../../.nuget/packages/system.reactive.linq/6.0.1-preview.1/lib/netstandard2.0/System.Reactive.Linq.dll"
#r @"../../../../../../../.nuget/packages/argu/6.1.1/lib/netstandard2.0/Argu.dll"
#r @"../../../../../../../.nuget/packages/system.commandline/2.0.0-beta4.22272.1/lib/net6.0/System.CommandLine.dll"
#r @"../../../../../../../.nuget/packages/asyncio/0.1.69/lib/netstandard2.0/AsyncIO.dll"
#r @"../../../../../../../.nuget/packages/netmq/4.0.1.12/lib/netstandard2.1/NetMQ.dll"

In [ ]:
#!import ../nbs/Common.fs
#!import ../nbs/Async.fs
#!import ../nbs/AsyncSeq.fs
#!import ../nbs/Networking.fs
#!import ../nbs/Runtime.fs
#!import ../nbs/FileSystem.fs

In [ ]:
open Common
open FileSystem

## sendJson

In [ ]:
let inline sendJson (port : int) (json : string) = async {
    let! portOpen = Networking.testPortOpen port
    if portOpen then
        // use runtime = new NetMQ.NetMQRuntime ()
        use request = new NetMQ.Sockets.RequestSocket ()
        request.Connect $"tcp://127.0.0.1:{port}"

        let msg = NetMQ.NetMQMessage ()
        msg.Append json

        NetMQ.OutgoingSocketExtensions.SendMultipartMessage (request, msg)
        let result = NetMQ.ReceivingSocketExtensions.ReceiveMultipartMessage (request, 10)
        // let! result = NetMQ.AsyncReceiveExtensions.ReceiveMultipartMessageAsync (request, 10) |> Async.AwaitTask
        let result = result |> Seq.toList |> List.map (fun x -> x.ConvertToString ())
        trace Debug (fun () -> $"sendJson / port: {port} / json: {json} / result: {result}") getLocals
        return true
    else
        trace Debug (fun () -> "sendJson / error: port not open") getLocals
        return false
}

## sendObj

In [ ]:
let inline sendObj port obj =
    let json = System.Text.Json.JsonSerializer.Serialize obj
    sendJson port json

## compile

In [ ]:
let inline compileFile timeout filePath = async {
    let fullPath = filePath |> System.IO.Path.GetFullPath
    let fileDir = fullPath |> System.IO.Path.GetDirectoryName
    let fileName = fullPath |> System.IO.Path.GetFileNameWithoutExtension
    let! code = fullPath |> System.IO.File.ReadAllTextAsync |> Async.AwaitTask

    let stream, disposable = FileSystem.watchDirectory true fileDir

    let port =
        if fileDir |> String.startsWith (System.IO.Path.GetTempPath ())
        then 13807
        else 13805

    let! ct = Async.CancellationToken
    let compiler = MailboxProcessor.Start (fun inbox -> async {
        let! availablePort = Networking.getAvailablePort (Some 60) port
        if availablePort <> port then
            let pingObj = {| Ping = true |}
            let! pingResult = pingObj |> sendObj port
            inbox.Post ()
        else
            let compilerPath =
                "../../deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release"
                |> System.IO.Path.GetFullPath

            let dllPath = compilerPath </> "Spiral.dll"
            // let commandsPath = compilerPath </> "compiler/supervisor/commands"

            let! exitCode, result =
                Runtime.executeWithOptionsAsync
                    {
                        Command = $@"dotnet ""{dllPath}"" port={port}"
                        CancellationToken = None
                        WorkingDirectory = None
                        OnLine = Some <| fun { Line = line } -> async {
                            if line |> String.contains $"Server bound to: tcp://*:{port}"
                            then inbox.Post ()
                        }
                    }
            trace Debug (fun () -> $"startSupervisor / exitCode: {exitCode} / result: {result}") getLocals
    }, ct)

    do! compiler.Receive ()

    let! fsxContentChild =
        stream
        |> FSharp.Control.AsyncSeq.choose (function
            | _, FileSystemChange.Changed (path, Some content) when path = $"{fileName}.fsx" -> Some content
            | _ -> None
        )
        |> FSharp.Control.AsyncSeq.tryFirst
        |> Async.runWithTimeoutAsync timeout
        |> Async.StartChild

    let inline getFileUri path =
        $"file:///{path |> String.trimStart [| '/' |]}"


    let fileOpenObj = {| FileOpen = {| uri = fullPath |> getFileUri; spiText = code |} |}
    let! fileOpenResult = fileOpenObj |> sendObj port

    let buildFileObj = {| BuildFile = {| uri = fullPath |> getFileUri; backend = "Fsharp" |} |}
    let! buildFileResult = buildFileObj |> sendObj port

    let! fsxContent = fsxContentChild

    disposable.Dispose ()

    return fsxContent |> Option.flatten |> Option.map (String.replace "\r\n" "\n")
}

In [ ]:
let inline compileCode timeout code = async {
    let tempDir = FileSystem.createTempDirectory ()

    let mainPath = tempDir </> "main.spi"
    do! System.IO.File.WriteAllTextAsync (mainPath, code) |> Async.AwaitTask

    let repositoryRoot = FileSystem.getSourceDirectory () |> FileSystem.findParent ".paket" false

    let spiprojPath = tempDir </> "package.spiproj"
    let spiprojCode =
        $"""packageDir: {repositoryRoot </> "spiral"}
packages:
    |core-
    fsharp
modules:
    main
"""
    do! System.IO.File.WriteAllTextAsync (spiprojPath, spiprojCode) |> Async.AwaitTask

    let! result = mainPath |> compileFile timeout

    do! tempDir |> FileSystem.deleteDirectoryAsync |> Async.Ignore

    return result
}

In [ ]:
//// test

"inl app () =
    0i32

inl main () =
    app
    |> dyn
    |> ignore
"
|> compileCode 8000
|> Async.runWithTimeout 8000
|> Option.flatten
|> _equal (Some "let rec closure0 () () : int32 =
    0
let v0 : (unit -> int32) = closure0()
()
")

05:20:17 #1 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230802-0520-1730-3092-3ab1c9dfa33d / result: { CreationTime = 2023-08-02 5:20:17 AM
  Exists = true }
05:20:17 #2 [Debug] runWithTimeoutAsync / timeout: 60
05:20:17 #3 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = None
  OnLine = Some <fun:it@11-242> }
05:20:17 #4 [Debug]  4628: Server bound to: tcp://*:13807 & tcp://*:13808
05:20:17 #5 [Debug]  4628: pwd: C:\home\git\polyglot\apps\spiral
05:20:17 #6 [Debug]  4628: dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
05:20:18 #7 [Debug] sendJson / port: 13807 / json: {"FileOpen":{"spiText":"inl app () =\n    0i32\n\ninl main () =\n    app\n    |\u003E dyn\n    |\u003E ignore\n

In [ ]:
//// test

"""inl app () =
    fsharp.console.write_line "error"
    1i32

inl main () =
    app
    |> dyn
    |> ignore
"""
|> compileCode 8000
|> Async.runWithTimeout 8000
|> Option.flatten
|> _equal (Some """let rec closure0 () () : int32 =
    let v0 : string = "error"
    System.Console.WriteLine v0
    1
let v0 : (unit -> int32) = closure0()
()
""")

05:20:22 #11 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230802-0520-2263-6304-6683fe21da16 / result: { CreationTime = 2023-08-02 5:20:22 AM
  Exists = true }
05:20:22 #12 [Debug] runWithTimeoutAsync / timeout: 60
05:20:22 #13 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = None
  OnLine = Some <fun:it@12-635> }
05:20:23 #14 [Debug]  4860: Server bound to: tcp://*:13807 & tcp://*:13808
05:20:23 #15 [Debug]  4860: pwd: C:\home\git\polyglot\apps\spiral
05:20:23 #16 [Debug]  4860: dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
05:20:23 #17 [Debug] sendJson / port: 13807 / json: {"FileOpen":{"spiText":"inl app () =\n    fsharp.console.write_line \u0022error\u0022\n    1i32\n\ninl m

## Arguments

In [ ]:
[<RequireQualifiedAccess>]
type Arguments =
    | [<Argu.ArguAttributes.Mandatory>] BuildFile of string * string
    | Timeout of int

    interface Argu.IArgParserTemplate with
        member s.Usage =
            match s with
            | BuildFile _ -> nameof Arguments.BuildFile
            | Timeout _ -> nameof Arguments.Timeout

## main

In [ ]:

let main args =
    let argsMap = args |> Runtime.parseArgsMap<Arguments>

    let inputPath, outputPath =
        match argsMap.[nameof Arguments.BuildFile] with
        | [ Arguments.BuildFile (inputPath, outputPath) ] -> Some (inputPath, outputPath)
        | _ -> None
        |> Option.get

    let timeout =
        match argsMap |> Map.tryFind (nameof Arguments.Timeout) with
        | Some [ Arguments.Timeout timeout ] -> timeout
        | _ -> 30000

    async {
        let! outputCode = inputPath |> compileFile timeout
        match outputCode with
        | Some outputCode ->
            do! System.IO.File.WriteAllTextAsync (outputPath, outputCode) |> Async.AwaitTask
            return 0
        | None ->
            return 1
    }
    |> Async.runWithTimeout timeout
    |> Option.defaultValue 1

In [ ]:
//// test

let args =
    System.Environment.GetEnvironmentVariable "ARGS"
    |> Runtime.splitArgs
    |> Seq.toArray

match args with
| [||] -> 0
| args -> if main args = 0 then 0 else failwith "main failed"

0